In [1]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import numpy as np

In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
path =r"C:\Users\NT550-052\OneDrive\바탕 화면\ASAC\프로젝트\ML\web_crawling\CGW\finaldata\finaldata_gw.csv"
data = pd.read_csv(path,sep="\t")
data = data["genre"].values

In [4]:
data = data.tolist()

In [369]:
# data = list(set(data))

In [5]:
data

['Indie, Simulation, Strategy',
 'Strategy',
 'Action, Adventure, Indie',
 'Action, Adventure, Indie',
 'Adventure, Indie',
 'Action, Indie',
 'Action, Indie',
 'Adventure, Indie',
 'Adventure, RPG, Strategy',
 'Action, Adventure, Indie, RPG, Simulation, Early Access',
 'Casual, Indie',
 'Adventure, Indie, RPG',
 'Action, Adventure, Indie, Simulation',
 'Adventure, Indie',
 'Adventure, Free to Play, Indie',
 'Adventure, Casual, Indie, RPG',
 'Action, Free to Play, Indie',
 'Action, Adventure, RPG',
 'Casual, Indie',
 'Adventure, Casual, Simulation',
 'Action, Indie, RPG, Simulation, Strategy',
 'Adventure, Indie',
 'Casual, Indie',
 'Action, Indie, Sports',
 'Casual',
 'Indie, RPG',
 'Casual, Free to Play, Indie',
 'Adventure',
 'Adventure',
 'Action, Indie',
 'Adventure',
 'Action, Adventure, Casual, Free to Play, Indie, Simulation, Early Access',
 'Simulation',
 'Casual, Indie, Simulation',
 'Action, Casual, Free to Play, Indie, Simulation',
 'Action, Indie, Early Access',
 'Indie, S

In [6]:
for i in range(len(data)):
    data[i] = data[i].split(",")
    for j in range(len(data[i])):
        if " " in data[i][j]:
            data[i][j]=data[i][j].replace(" ","")
    data[i] = ",".join(data[i])
data

['Indie,Simulation,Strategy',
 'Strategy',
 'Action,Adventure,Indie',
 'Action,Adventure,Indie',
 'Adventure,Indie',
 'Action,Indie',
 'Action,Indie',
 'Adventure,Indie',
 'Adventure,RPG,Strategy',
 'Action,Adventure,Indie,RPG,Simulation,EarlyAccess',
 'Casual,Indie',
 'Adventure,Indie,RPG',
 'Action,Adventure,Indie,Simulation',
 'Adventure,Indie',
 'Adventure,FreetoPlay,Indie',
 'Adventure,Casual,Indie,RPG',
 'Action,FreetoPlay,Indie',
 'Action,Adventure,RPG',
 'Casual,Indie',
 'Adventure,Casual,Simulation',
 'Action,Indie,RPG,Simulation,Strategy',
 'Adventure,Indie',
 'Casual,Indie',
 'Action,Indie,Sports',
 'Casual',
 'Indie,RPG',
 'Casual,FreetoPlay,Indie',
 'Adventure',
 'Adventure',
 'Action,Indie',
 'Adventure',
 'Action,Adventure,Casual,FreetoPlay,Indie,Simulation,EarlyAccess',
 'Simulation',
 'Casual,Indie,Simulation',
 'Action,Casual,FreetoPlay,Indie,Simulation',
 'Action,Indie,EarlyAccess',
 'Indie,Strategy',
 'Indie,Simulation',
 'Action,Indie,Strategy',
 'Adventure,Casual,

In [372]:
# data = [
#     "action","arcade & rhythm","fighting & martial arts", "first-person shooter","hack & slash","platformer & runner",
#     "third-person shooter", "shmup","role-playing","action rpg","adventure rpg","jrpg","party-based","rogue-like","strategy rpg",
#     "turn-based","strategy","card & board","city & settlement","grand & 4x","military","real-time strategy","tower defense","turn-based strategy",
#     "adventure","adventure rpg","casual","hidden object","metroidvania","puzzle","story-rich","visual novel","simulation","building & automation",
#     "dating","farming & crafting","hobby & job","life & immersive","sandbox & physics","space & flight","sports & racing","all sports","fishing & hunting",
#     "individual sports","racing","racing sim","sports sim","team sports"
# ]
# data = list(set(data))

In [7]:
normalized_text = []
for string in data:
     tokens = re.sub(r"[^a-z0-9 ]+", " ", string.lower())
     normalized_text.append(tokens)
print(normalized_text)
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]
# result = [[sentence] for sentence in normalized_text]

['indie simulation strategy', 'strategy', 'action adventure indie', 'action adventure indie', 'adventure indie', 'action indie', 'action indie', 'adventure indie', 'adventure rpg strategy', 'action adventure indie rpg simulation earlyaccess', 'casual indie', 'adventure indie rpg', 'action adventure indie simulation', 'adventure indie', 'adventure freetoplay indie', 'adventure casual indie rpg', 'action freetoplay indie', 'action adventure rpg', 'casual indie', 'adventure casual simulation', 'action indie rpg simulation strategy', 'adventure indie', 'casual indie', 'action indie sports', 'casual', 'indie rpg', 'casual freetoplay indie', 'adventure', 'adventure', 'action indie', 'adventure', 'action adventure casual freetoplay indie simulation earlyaccess', 'simulation', 'casual indie simulation', 'action casual freetoplay indie simulation', 'action indie earlyaccess', 'indie strategy', 'indie simulation', 'action indie strategy', 'adventure casual indie rpg', 'casual indie', 'indie simu

In [8]:
result

[['indie', 'simulation', 'strategy'],
 ['strategy'],
 ['action', 'adventure', 'indie'],
 ['action', 'adventure', 'indie'],
 ['adventure', 'indie'],
 ['action', 'indie'],
 ['action', 'indie'],
 ['adventure', 'indie'],
 ['adventure', 'rpg', 'strategy'],
 ['action', 'adventure', 'indie', 'rpg', 'simulation', 'earlyaccess'],
 ['casual', 'indie'],
 ['adventure', 'indie', 'rpg'],
 ['action', 'adventure', 'indie', 'simulation'],
 ['adventure', 'indie'],
 ['adventure', 'freetoplay', 'indie'],
 ['adventure', 'casual', 'indie', 'rpg'],
 ['action', 'freetoplay', 'indie'],
 ['action', 'adventure', 'rpg'],
 ['casual', 'indie'],
 ['adventure', 'casual', 'simulation'],
 ['action', 'indie', 'rpg', 'simulation', 'strategy'],
 ['adventure', 'indie'],
 ['casual', 'indie'],
 ['action', 'indie', 'sports'],
 ['casual'],
 ['indie', 'rpg'],
 ['casual', 'freetoplay', 'indie'],
 ['adventure'],
 ['adventure'],
 ['action', 'indie'],
 ['adventure'],
 ['action',
  'adventure',
  'casual',
  'freetoplay',
  'indie',

In [9]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 46252


In [10]:
from gensim.models import FastText
model = FastText(result,min_count=2,workers=4, sg=1)
model.build_vocab(corpus_iterable=result)
model.train(corpus_iterable=result, total_examples=len(result), epochs=10)

c:\Users\NT550-052\anaconda3\envs\mlp\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


(150847, 1282930)

In [14]:
model.wv["indie"]

array([-7.46909529e-02,  7.03254044e-02, -9.27907377e-02,  3.23317528e-01,
        1.25612944e-01,  2.72472277e-02, -5.20800948e-02,  4.20904517e-01,
        1.29500970e-01,  3.33591670e-01, -1.74758639e-02,  2.33932406e-01,
        1.98906764e-01, -7.90092126e-02, -3.53001058e-01,  3.84184957e-01,
        3.30556601e-01,  6.87717423e-02,  3.73104028e-02, -3.33778225e-02,
        1.65897906e-01,  2.32884604e-02, -7.11123794e-02,  3.61238688e-01,
       -3.20638567e-01, -5.39362542e-02,  6.09579869e-02, -2.42967993e-01,
       -2.14238659e-01, -1.37357518e-01, -4.47223753e-01, -4.69255000e-02,
       -2.83468068e-01, -3.66155297e-01, -2.25681335e-01, -2.66029745e-01,
        2.33465925e-01, -1.58776775e-01,  2.26485550e-01, -2.77415901e-01,
       -2.31091127e-01, -1.45617465e-03, -2.56702065e-01, -1.13006800e-01,
       -2.95671895e-02,  2.65101284e-01, -2.36901805e-01,  2.70669878e-01,
       -1.57925799e-01,  2.19243512e-01, -7.93632045e-02,  2.29475483e-01,
       -1.18011035e-01,  

In [15]:
model.wv["indie"].shape

(100,)

In [16]:
len(model.wv.key_to_index)

30

In [384]:
model.wv.most_similar("casual")

[('earlyaccess', 0.7563517093658447),
 ('gore', 0.6510389447212219),
 ('sexualcontent', 0.6224485039710999),
 ('gamedevelopment', 0.6162863969802856),
 ('design', 0.6113567352294922),
 ('nudity', 0.6112880110740662),
 ('modeling', 0.6070531606674194),
 ('utilities', 0.6050287485122681),
 ('photoediting', 0.6042643189430237),
 ('webpublishing', 0.6041508913040161)]

In [385]:
# 단어 유사도
model.wv.similarity('adventure indie', 'casual indie')

0.45492953

In [386]:
from sklearn.cluster import KMeans

word_vectors = model.wv.vectors # 어휘의 feature vector


In [387]:
word_vectors.shape

(30, 100)

In [401]:
num_clusters = int(word_vectors.shape[0]/3) # 어휘 크기의 1/5나 평균 5단어
print(num_clusters)
num_clusters = int(num_clusters)

10


In [402]:
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

idx = list(idx)
names = model.wv.index_to_key
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}

c:\Users\NT550-052\anaconda3\envs\mlp\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\NT550-052\anaconda3\envs\mlp\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [403]:
clt = {}
for c in range(num_clusters):
    # 클러스터 번호를 출력
    print("\ncluster {}".format(c))
    
    words = []
    cluster_values = list(word_centroid_map.values())
    for i in range(len(cluster_values)):
        if (cluster_values[i] == c):
            words.append(list(word_centroid_map.keys())[i])    
    clt[str(c)] = words        
    print(words)


cluster 0
['massivelymultiplayer']

cluster 1
['violent', 'gore', 'nudity', 'sexualcontent', 'gamedevelopment', 'accounting']

cluster 2
['education', 'utilities', 'design', 'illustration', 'animation', 'modeling', 'softwaretraining', 'audioproduction', 'webpublishing', 'videoproduction', 'photoediting']

cluster 3
['action', 'strategy']

cluster 4
['casual', 'rpg', 'earlyaccess', 'n', 'freetoplay']

cluster 5
['indie']

cluster 6
['sports']

cluster 7
['racing']

cluster 8
['simulation']

cluster 9
['adventure']


In [404]:
clt

{'0': ['massivelymultiplayer'],
 '1': ['violent',
  'gore',
  'nudity',
  'sexualcontent',
  'gamedevelopment',
  'accounting'],
 '2': ['education',
  'utilities',
  'design',
  'illustration',
  'animation',
  'modeling',
  'softwaretraining',
  'audioproduction',
  'webpublishing',
  'videoproduction',
  'photoediting'],
 '3': ['action', 'strategy'],
 '4': ['casual', 'rpg', 'earlyaccess', 'n', 'freetoplay'],
 '5': ['indie'],
 '6': ['sports'],
 '7': ['racing'],
 '8': ['simulation'],
 '9': ['adventure']}